<a href="https://colab.research.google.com/github/koushikgptREAL/ANN-MOBILE-CLASSIFIER-WOA/blob/main/NeuralNetworks_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

# Load data
dataset = pd.read_csv('/content/test.csv', encoding='ISO-8859-1')

# Binary classification target
dataset['High_End'] = dataset['ram'].apply(lambda x: 1 if x > 3000 else 0)

# Select features
X = dataset.drop(columns=['High_End']).values
y = dataset['High_End'].values

# Handle missing values
X = SimpleImputer(strategy='mean').fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [ ]:
def build_and_train_ann(X_train, X_test, y_train, y_test, lr, neurons, batch_size, epochs=30, verbose=0):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=neurons, activation='relu'))
    model.add(tf.keras.layers.Dense(units=neurons, activation='relu'))
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, batch_size=16, epochs=10, verbose=1)
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred)
    return acc


In [ ]:
import random

def whale_optimization(X_train, X_test, y_train, y_test, num_agents=5, max_iter=10):
    dim = 3
    lb = [4, 0.0001, 8]
    ub = [64, 0.1, 128]

    def fitness(position):
        neurons = int(position[0])
        lr = float(position[1])
        batch_size = int(position[2])
        return build_and_train_ann(X_train, X_test, y_train, y_test, lr, neurons, batch_size)

    # Initialize positions
    positions = np.array([np.random.uniform(lb, ub) for _ in range(num_agents)])
    best_score = -np.inf
    best_position = None

    for t in range(max_iter):
        a = 2 - t * (2 / max_iter)
        for i in range(num_agents):
            r = random.random()
            A = 2 * a * r - a
            C = 2 * r
            p = random.random()
            b = 1
            l = random.uniform(-1, 1)

            if p < 0.5:
                if abs(A) < 1 and best_position is not None:
                    D = abs(C * best_position - positions[i])
                    positions[i] = best_position - A * D
                else:
                    rand_idx = random.randint(0, num_agents - 1)
                    D = abs(C * positions[rand_idx] - positions[i])
                    positions[i] = positions[rand_idx] - A * D
            else:
                if best_position is not None:
                    D = abs(best_position - positions[i])
                    positions[i] = D * np.exp(b * l) * np.cos(2 * np.pi * l) + best_position

            # Clamp values within bounds
            positions[i] = np.clip(positions[i], lb, ub)

            # Evaluate fitness
            score = fitness(positions[i])
            if score > best_score:
                best_score = score
                best_position = positions[i]

        print(f"Iteration {t+1}, Best Accuracy: {best_score:.4f}")

    return best_position, best_score


In [ ]:
best_params, best_accuracy = whale_optimization(X_train, X_test, y_train, y_test, num_agents=3, max_iter=4)
neurons = int(best_params[0])
lr = float(best_params[1])
batch_size = int(best_params[2])
print(f"Best Parameters - Neurons: {neurons}, LR: {lr:.5f}, Batch Size: {batch_size}")
print(f"Best Accuracy: {best_accuracy:.4f}")


Epoch 1/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6043 - loss: 0.7554
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8245 - loss: 0.4636
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8953 - loss: 0.2853
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9453 - loss: 0.1692
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9783 - loss: 0.0919
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9773 - loss: 0.0676
Epoch 7/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9917 - loss: 0.0528
Epoch 8/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9871 - loss: 0.0511
Epoch 9/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9838 - loss: 0.0426
Epoch 10/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9986 - loss: 0.0277
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Epoch 1/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7252 - loss: 0.6926
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━